In [1]:
import pandas as pd
from sklearn.model_selection._split import _BaseKFold
from sklearn.utils import check_random_state
import scipy.sparse as sp
import itertools
import numpy as np
import random

seed = 250

random.seed(seed)
np.random.seed(seed)

In [2]:
def _fold_tie_break(desired_samples_per_fold, M):
    """Helper function to split a tie between folds with same desirability of a given sample

    Parameters
    ----------
    desired_samples_per_fold: np.array[Float], :code:`(n_splits)`
        number of samples desired per fold
    M : np.array(int)
        List of folds between which to break the tie

    Returns
    -------
    fold_number : int
        The selected fold index to put samples into
    """
    if len(M) == 1:
        return M[0]
    else:
        max_val = max(desired_samples_per_fold[M])
        M_prim = np.where(
            np.array(desired_samples_per_fold) == max_val)[0]
        M_prim = np.array([x for x in M_prim if x in M])
        return np.random.choice(M_prim, 1)[0]


def _get_most_desired_combination(samples_with_combination):
    """Select the next most desired combination whose evidence should be split among folds

    Parameters
    ----------
    samples_with_combination : Dict[Combination, List[int]], :code:`(n_combinations)`
            map from each label combination present in y to list of sample indexes that have this combination assigned

    Returns
    -------
    combination: Combination
        the combination to split next
    """
    currently_chosen = None
    best_number_of_combinations, best_support_size = None, None

    for combination, evidence in samples_with_combination.items():
        number_of_combinations, support_size = (len(set(combination)), len(evidence))
        if support_size == 0:
            continue
        if currently_chosen is None or (
                best_number_of_combinations < number_of_combinations and best_support_size > support_size
        ):
            currently_chosen = combination
            best_number_of_combinations, best_support_size = number_of_combinations, support_size

    return currently_chosen

class IterativeStratification(_BaseKFold):
    """Iteratively stratify a multi-label data set into folds

    Construct an interative stratifier that splits the data set into folds trying to maintain balanced representation
    with respect to order-th label combinations.

    Attributes
    ----------

    n_splits : number of splits, int
        the number of folds to stratify into

    order : int, >= 1
        the order of label relationship to take into account when balancing sample distribution across labels

    sample_distribution_per_fold : None or List[float], :code:`(n_splits)`
        desired percentage of samples in each of the folds, if None and equal distribution of samples per fold
        is assumed i.e. 1/n_splits for each fold. The value is held in :code:`self.percentage_per_fold`.

    random_state : int
        the random state seed (optional)
    """

    def __init__(self, n_splits=3, order=1, sample_distribution_per_fold = None, random_state=None):
        self.order = order
        super(
            IterativeStratification,
            self).__init__(n_splits,
                           shuffle=True,
                           random_state=random_state)

        if sample_distribution_per_fold:
            self.percentage_per_fold = sample_distribution_per_fold
        else:
            self.percentage_per_fold = [1 / float(self.n_splits) for _ in range(self.n_splits)]

    def _prepare_stratification(self, y):
        """Prepares variables for performing stratification

        For the purpose of clarity, the type Combination denotes List[int], :code:`(self.order)` and represents a
        label combination of the order we want to preserve among folds in stratification. The total number of
        combinations present in :code:`(y)` will be denoted as :code:`(n_combinations)`.

        Sets
        ----

        self.n_samples, self.n_labels : int, int
            shape of y

        self.desired_samples_per_fold: np.array[Float], :code:`(n_splits)`
            number of samples desired per fold

        self.desired_samples_per_combination_per_fold: Dict[Combination, np.array[Float]], :code:`(n_combinations, n_splits)`
            number of samples evidencing each combination desired per each fold

        Parameters
        ----------

        y : output matrix or array of arrays (n_samples, n_labels)

        Returns
        -------

        rows : List[List[int]], :code:`(n_samples, n_labels)`
            list of label indices assigned to each sample

        rows_used : Dict[int, bool], :code:`(n_samples)`
            boolean map from a given sample index to boolean value whether it has been already assigned to a fold or not

        all_combinations :  List[Combination], :code:`(n_combinations)`
            list of all label combinations of order self.order present in y

        per_row_combinations : List[Combination], :code:`(n_samples)`
            list of all label combinations of order self.order present in y per row

        samples_with_combination : Dict[Combination, List[int]], :code:`(n_combinations)`
            map from each label combination present in y to list of sample indexes that have this combination assigned

        folds: List[List[int]] (n_splits)
            list of lists to be populated with samples

        """
        self.n_samples, self.n_labels = y.shape
        self.desired_samples_per_fold = np.array([self.percentage_per_fold[i] * self.n_samples
                                                  for i in range(self.n_splits)])
        rows = sp.lil_matrix(y).rows
        rows_used = {i: False for i in range(self.n_samples)}
        all_combinations = []
        per_row_combinations = [[] for i in range(self.n_samples)]
        samples_with_combination = {}
        folds = [[] for _ in range(self.n_splits)]

        # for every row
        for sample_index, label_assignment in enumerate(rows):
            # for every n-th order label combination
            # register combination in maps and lists used later
            for combination in itertools.combinations_with_replacement(label_assignment, self.order):
                if combination not in samples_with_combination:
                    samples_with_combination[combination] = []

                samples_with_combination[combination].append(sample_index)
                all_combinations.append(combination)
                per_row_combinations[sample_index].append(combination)

        all_combinations = [list(x) for x in set(all_combinations)]

        self.desired_samples_per_combination_per_fold = {
            combination:
                np.array([len(evidence_for_combination) * self.percentage_per_fold[j]
                          for j in range(self.n_splits)])
            for combination, evidence_for_combination in samples_with_combination.items()
        }
        return rows, rows_used, all_combinations, per_row_combinations, samples_with_combination, folds

    def _distribute_positive_evidence(self, rows_used, folds, samples_with_combination, per_row_combinations):
        """Internal method to distribute evidence for labeled samples across folds

        For params, see documentation of :code:`self._prepare_stratification`. Does not return anything,
        modifies params.
        """
        l = _get_most_desired_combination(samples_with_combination)
        while l is not None:
            while len(samples_with_combination[l]) > 0:
                row = samples_with_combination[l].pop()
                if rows_used[row]:
                    continue

                max_val = max(self.desired_samples_per_combination_per_fold[l])
                M = np.where(
                    np.array(self.desired_samples_per_combination_per_fold[l]) == max_val)[0]
                m = _fold_tie_break(self.desired_samples_per_combination_per_fold[l], M)
                folds[m].append(row)
                rows_used[row] = True
                for i in per_row_combinations[row]:
                    if row in samples_with_combination[i]:
                        samples_with_combination[i].remove(row)
                    self.desired_samples_per_combination_per_fold[i][m] -= 1
                self.desired_samples_per_fold[m] -= 1

            l = _get_most_desired_combination(samples_with_combination)

    def _distribute_negative_evidence(self, rows_used, folds):
        """Internal method to distribute evidence for unlabeled samples across folds

        For params, see documentation of :code:`self._prepare_stratification`. Does not return anything,
        modifies params.
        """
        available_samples = [
            i for i, v in rows_used.items() if not v]
        samples_left = len(available_samples)

        while samples_left > 0:
            row = available_samples.pop()
            rows_used[row] = True
            samples_left -= 1
            fold_selected = np.random.choice(np.where(self.desired_samples_per_fold > 0)[0], 1)[0]
            self.desired_samples_per_fold[fold_selected] -= 1
            folds[fold_selected].append(row)

    def _iter_test_indices(self, X, y=None, groups=None):
        """Internal method for providing scikit-learn's split with folds

        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples,)
            The target variable for supervised learning problems.
            Stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.

        Yields
        ------
        fold : List[int]
            indexes of test samples for a given fold, yielded for each of the folds
        """
        if self.random_state:
            check_random_state(self.random_state)

        rows, rows_used, all_combinations, per_row_combinations, samples_with_combination, folds = \
            self._prepare_stratification(y)

        self._distribute_positive_evidence(rows_used, folds, samples_with_combination, per_row_combinations)
        self._distribute_negative_evidence(rows_used, folds)

        for fold in folds:
            yield fold

In [3]:
def iterative_train_test_split(X, y, test_size, random_state):
    """Iteratively stratified train/test split

    Parameters
    ----------
    test_size : float, [0,1]
        the proportion of the dataset to include in the test split, the rest will be put in the train set

    Returns
    -------
    X_train, y_train, X_test, y_test
        stratified division into train/test split
    """

    stratifier = IterativeStratification(n_splits=2, order=2, sample_distribution_per_fold=[test_size, 1.0-test_size], random_state=random_state)
    train_indexes, test_indexes = next(stratifier.split(X, y))

    X_train, y_train = X[train_indexes, :], y[train_indexes, :]
    X_test, y_test = X[test_indexes, :], y[test_indexes, :]

    return X_train, y_train, X_test, y_test

In [4]:
patients = pd.read_hdf('MIMIC_EXTRACT_all_hourly_data.h5', key='/patients')

In [5]:
gender_mapping = {
           'M': 1,
           'F': 0}

patients['gender'] = patients['gender'].map(gender_mapping)

In [6]:
labs_mean = pd.read_hdf('MIMIC_EXTRACT_all_hourly_data.h5', key='/vitals_labs_mean')

In [7]:
interventions = pd.read_hdf('MIMIC_EXTRACT_all_hourly_data.h5', key='/interventions')

In [8]:
interventions = interventions.droplevel(['hadm_id'])

In [9]:
with pd.HDFStore('MIMIC_EXTRACT_all_hourly_data.h5') as hdf:
    # This prints a list of all group names:
    print(hdf.keys())

['/codes', '/interventions', '/patients', '/vitals_labs', '/vitals_labs_mean', '/patients/meta/values_block_6/meta', '/patients/meta/values_block_5/meta', '/patients/meta/values_block_4/meta', '/patients/meta/values_block_0/meta']


In [10]:
# Select the ones you want
patients = patients[['gender','age', 'ethnicity', 'diagnosis_at_admission', 'mort_icu', 'mort_hosp', 'readmission_30']]

In [11]:
phenotypes = pd.read_csv('phenotypes.csv')

In [12]:
phenotypes.loc[phenotypes['Acute myocardial infarction'] == 1].index

Index([   19,    52,    58,    70,    71,    73,    87,    94,    97,   118,
       ...
       41780, 41784, 41786, 41801, 41802, 41803, 41842, 41888, 41891, 41892],
      dtype='int64', length=4337)

In [13]:
phenotypes.set_index(['subject_id', 'icustay_id'], drop=True, inplace=True)

In [14]:
patients = patients.droplevel('hadm_id')

In [15]:
df1 = patients.join(phenotypes, on=['subject_id','icustay_id'])

In [16]:
# Select the ones you want
df1 = df1[['gender', 'age', 'mort_icu', 'mort_hosp', 'readmission_30', 'Shock', 'Acute cerebrovascular disease', 'Acute myocardial infarction',
       'Cardiac dysrhythmias', 'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
          'Congestive heart failure- nonhypertensive']]

In [17]:
# df1 = df1.droplevel('icustay_id')

In [18]:
# df1.index.has_duplicates

In [19]:
labs_mean = labs_mean.droplevel('hadm_id')

In [20]:
# labs_mean = labs_mean.droplevel('subject_id')

In [21]:
labs_mean.update(labs_mean.groupby(level=0).ffill())

In [22]:
labs_mean2 = labs_mean.droplevel([1], axis=1)

In [23]:
labs_mean2

LEVEL2                          alanine aminotransferase  albumin  \
subject_id icustay_id hours_in                                      
3          211552     0                             25.0      1.8   
                      1                             25.0      1.8   
                      2                             25.0      1.8   
                      3                             25.0      1.8   
                      4                             25.0      1.8   
...                                                  ...      ...   
99999      246512     22                             NaN      NaN   
                      23                             NaN      NaN   
                      24                             NaN      NaN   
                      25                             NaN      NaN   
                      26                             NaN      NaN   

LEVEL2                          albumin ascites  albumin pleural  \
subject_id icustay_id hours_in                                     
3          211552     0                     NaN              NaN   
                      1                     NaN              NaN   
                      2                     NaN              NaN   
                      3                     NaN              NaN   
                      4                     NaN              NaN   
...                                         ...              ...   
99999      246512     22                    NaN              NaN   
                      23                    NaN              NaN   
                      24                    NaN              NaN   
                      25                    NaN              NaN   
                      26                    NaN              NaN   

LEVEL2                          albumin urine  alkaline phosphate  anion gap  \
subject_id icustay_id hours_in                                                 
3          211552     0                   NaN                73.0  20.666667   
                      1                   NaN                73.0  20.666667   
                      2                   NaN                73.0  20.666667   
                      3                   NaN                73.0  20.666667   
                      4                   NaN                73.0  20.666667   
...                                       ...                 ...        ...   
99999      246512     22                  NaN                 NaN  10.000000   
                      23                  NaN                 NaN  10.000000   
                      24                  NaN                 NaN  10.000000   
                      25                  NaN                 NaN  10.000000   
                      26                  NaN                 NaN  10.000000   

LEVEL2                          asparate aminotransferase  basophils  \
subject_id icustay_id hours_in                                         
3          211552     0                              69.0        NaN   
                      1                              69.0        NaN   
                      2                              69.0        NaN   
                      3                              69.0        NaN   
                      4                              69.0        NaN   
...                                                   ...        ...   
99999      246512     22                              NaN        NaN   
                      23                              NaN        NaN   
                      24                              NaN        NaN   
                      25                              NaN        NaN   
                      26                              NaN        NaN   

LEVEL2                          bicarbonate  ...  total protein  \
subject_id icustay_id hours_in               ...                  
3          211552     0           16.333333  ...            NaN   
                      1           16.3

In [271]:
# df1 = df1.droplevel('subject_id')

In [272]:
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 1000)  # or 199

# Missing values per feature
labs_mean2.isnull().sum(axis = 0)/2200954*100

LEVEL2
alanine aminotransferase                     50.714463
albumin                                      57.032996
albumin ascites                              99.648607
albumin pleural                              99.554693
albumin urine                                99.686636
alkaline phosphate                           51.770505
anion gap                                     6.078092
asparate aminotransferase                    50.770984
basophils                                    58.951982
bicarbonate                                   3.468132
bilirubin                                    51.010653
blood urea nitrogen                           3.123600
co2                                          46.410875
co2 (etco2, pco2, etc.)                      27.987909
calcium                                      14.954697
calcium ionized                              43.845487
calcium urine                                99.466595
cardiac index                                85.727416
car

In [273]:
labs_mean2 = labs_mean2[['anion gap', 'white blood cell count', 'weight', 'temperature', 'systolic blood pressure',
                        'sodium', 'respiratory rate', 'red blood cell count', 'prothrombin time pt', 'prothrombin time inr', 'potassium',
                        'platelets', 'phosphorous', 'phosphate', 'partial thromboplastin time', 'oxygen saturation', 'mean corpuscular hemoglobin concentration',
                        'magnesium', 'hemoglobin', 'hematocrit', 'heart rate', 'glucose', 'diastolic blood pressure', 'creatinine', 'chloride',
                        'calcium', 'blood urea nitrogen', 'bicarbonate']]

In [274]:
labs_mean2.update(labs_mean2.groupby(level=1).ffill())
labs_mean2.update(labs_mean2.groupby(level=1).bfill())

In [275]:
df2 = df1.merge(labs_mean2, left_index=True, right_index=True)

In [276]:
df2 = df2.merge(interventions, left_index=True, right_index=True)

In [277]:
df2 = df2.droplevel('icustay_id')

In [278]:
# Drop missing values samples
missing_samples = df2[df2.isnull().any(axis=1)].index.get_level_values(0).tolist()
df2.drop(missing_samples, level=0, axis=0, inplace=True) 

In [279]:
len(pd.unique(df2.index.get_level_values(0)))

17279

In [280]:
df2.columns

Index(['gender', 'age', 'mort_icu', 'mort_hosp', 'readmission_30', 'Shock',
       'Acute cerebrovascular disease', 'Acute myocardial infarction',
       'Cardiac dysrhythmias', 'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Congestive heart failure- nonhypertensive', 'anion gap',
       'white blood cell count', 'weight', 'temperature',
       'systolic blood pressure', 'sodium', 'respiratory rate',
       'red blood cell count', 'prothrombin time pt', 'prothrombin time inr',
       'potassium', 'platelets', 'phosphorous', 'phosphate',
       'partial thromboplastin time', 'oxygen saturation',
       'mean corpuscular hemoglobin concentration', 'magnesium', 'hemoglobin',
       'hematocrit', 'heart rate', 'glucose', 'diastolic blood pressure',
       'creatinine', 'chloride', 'calcium', 'blood urea nitrogen',
       'bicarbonate', 'vent', 'vaso', 'adenosine', 'dobutamine', 'dopamine',
       'epinephrine', 'isuprel', 'milrinone', '

In [281]:
y = df2[['mort_icu', 'mort_hosp', 'readmission_30', 'Shock',
       'Acute cerebrovascular disease', 'Acute myocardial infarction',
       'Cardiac dysrhythmias', 'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Congestive heart failure- nonhypertensive']]
y = y.droplevel(1)
y = y[~y.index.duplicated(keep='first')]

In [282]:
X = df2.drop(labels=['mort_icu', 'mort_hosp', 'readmission_30', 'Shock',
       'Acute cerebrovascular disease', 'Acute myocardial infarction',
       'Cardiac dysrhythmias', 'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Congestive heart failure- nonhypertensive'], axis=1, inplace=False)

In [283]:
X_ind = X.index.get_level_values(0)
X_ind = X_ind[~X_ind.duplicated(keep='first')]
X_ind = np.tile(X_ind, (2, 1))
X_ind = X_ind.transpose()

In [284]:
# Split the data into train and test

y = y.to_numpy()

X_train_ind, y_train, X_test_ind, y_test = iterative_train_test_split(X_ind, y, test_size = 0.2, random_state=seed)
X_train_ind, y_train, X_val_ind, y_val = iterative_train_test_split(X_train_ind, y_train, test_size = 0.2, random_state=seed)

In [285]:
X.reset_index(level=1, inplace=True)
X_train = X.loc[X_train_ind[:, 0].tolist()]
X_val = X.loc[X_val_ind[:, 0].tolist()]
X_test = X.loc[X_test_ind[:, 0].tolist()]

In [286]:
X_train.reset_index(inplace=True)
X_val.reset_index(inplace=True)
X_test.reset_index(inplace=True)

X_train.set_index(['subject_id', 'hours_in'], inplace=True)
X_val.set_index(['subject_id', 'hours_in'], inplace=True)
X_test.set_index(['subject_id', 'hours_in'], inplace=True)

In [287]:
# create time-series input for LSTM of shape [n, timestep, features]
def split_sequence(dataframe, n_steps):
    lstm_input = np.empty((len(dataframe.index.levels[0]), n_steps, 44))
    lstm_input[:] = np.nan
    for i in range(len(dataframe.index.levels[0])):
        sample = dataframe.loc[dataframe.index.levels[0][i].tolist()]
        sequence = sample.to_numpy()
        n_features = sequence.shape[1]
        time_length = sequence.shape[0]

        if n_steps > time_length:
            a = np.empty((n_steps-time_length,n_features))
            for j in range((n_steps-time_length)):
                a[j, :] = sequence[0, :]
            sequence = np.vstack((a,sequence))
        else: sequence = sequence[-n_steps:, :]
        lstm_input[i, :, :] = sequence
    
    return lstm_input

In [288]:
# Extract data into LSTM timeseries format with 24 1-hour timesteps
X_train = split_sequence(X_train, 24)
X_val = split_sequence(X_val, 24)
X_test = split_sequence(X_test, 24)

In [289]:
np.save('X_train_multilabel_full_MIMICX_250', X_train)
np.save('X_test_multilabel_full_MIMICX_250', X_test)
np.save('X_val_multilabel_full_MIMICX_250', X_val)
np.save('y_train_multilabel_full_MIMICX_250', y_train)
np.save('y_val_multilabel_full_MIMICX_250', y_val)
np.save('y_test_multilabel_full_MIMICX_250', y_test)

In [290]:
X_test.shape

(3456, 24, 44)

In [4]:
y = np.load('y_train_multilabel_full_MIMICX_250.npy')

In [5]:
y.value_counts()

AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [29]:
unique, counts = np.unique(y[:, 8], return_counts=True)
result = dict(zip(unique, counts/len(y)))

In [30]:
result

{0.0: 0.8818050280340025, 1.0: 0.11819497196599747}